# This is just to share what I have done in this competition. Kindly note that some code snippets were copies from other notebooks, unortunately I can't remeber the link to make the references.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import pickle
from datetime import datetime
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from keras.models import Model
import os
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import normalize
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf

In [ ]:
# Tiling made easy with this libarary
pip install patchify

In [ ]:
from PIL import Image, ImageDraw
from patchify import patchify
import tifffile as tiff
import gc
import json
import csv
import math, re, os
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
print("Tensorflow version " + tf.__version__)

In [ ]:
#pip install imgaug
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug import augmenters as iaa
from imgaug import parameters as iap
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

In [ ]:
def augment(image,mask):
    #image = m#ia.quokka(size=(128, 128), extract="square")
    n = mask
    segmap = (n>0).astype('uint8')
    segmap = SegmentationMapsOnImage(segmap, shape=image.shape)
    
    # We can do many transformations
    seq = iaa.Sequential([
        iaa.Dropout([0.05, 0.2]),      # drop 5% or 20% of all pixels
        iaa.Affine(rotate=(-45, 45)),  # rotate by -45 to 45 degrees (affects segmaps)
        iaa.ElasticTransformation(alpha=50, sigma=5),  # apply water effect (affects segmaps)
        iaa.GaussianBlur(sigma=iap.Uniform(0.9, 1.0)),
        iaa.MedianBlur(k=(3, 11))
    ], random_order=True)

    images_aug = []
    segmaps_aug = []
    #for _ in range(5):
    images_aug_i, segmaps_aug_i = seq(image=image, segmentation_maps=segmap)
    images_aug.append(images_aug_i)
    segmaps_aug.append(segmaps_aug_i)

    cells = []
    for image_aug, segmap_aug in zip(images_aug, segmaps_aug):
        cells.append(image)
        cells.append(n)
        cells.append(image_aug)
        cells.append(segmap_aug.draw(size=image_aug.shape[:2])[0])
    len(cells)

    #fig, axs = plt.subplots(1,4,figsize=(10,10))
    #axs[0].imshow(cells[0])
    #axs[1].imshow(cells[1])
    #axs[2].imshow(cells[2])
    #axs[3].imshow(cells[3])
    return cells[0],cells[1],cells[2],cells[3]

In [ ]:
# The tile sizes
size_x = 4000
size_y = 4000

In [ ]:
def file_list(folder):
    files_list = os.listdir('../input/hubmap-kidney-segmentation/'+folder)
    files = []
    for file in files_list:
        if ".tiff" in file:
            files.append(file.split(".")[0])
    return files

In [ ]:
# Create folder structure
try:
    !rm -r images
    !rm -r masks
    print("Deleted")
except:
    pass
!mkdir images
!mkdir masks
print("created")

In [ ]:
def read_tiff(file_id,folder):
    '''
        This funciton will take a tiff file id with its folder location and read it. It should note that format of tiff
        files are not same across the folder. Some tiff images has more dims than other and sometimes the color channel
        is located differently. This need to checked manually before creating this funcion.
    '''
    print("Reading file ",file_id,end="")
    large_image_stack = tiff.imread('../input/hubmap-kidney-segmentation/'+folder+'/'+file_id+'.tiff')#[0,0,:,:,:]
    print("Done")
    shape = large_image_stack.shape
    if(len(shape)>3):
        print("Image has more dims")
        large_image_stack = tiff.imread('../input/hubmap-kidney-segmentation/'+folder+'/'+file_id+'.tiff')[0,0,:,:,:]
    else:
        print("Image has 3 dims")
    shape = large_image_stack.shape
    if shape[0]==3:
        print("Image is channel first, converting")
        large_image_stack = np.einsum('ijk->jki',large_image_stack)
    else:
        print("Image is channel last, no need to convert")
    shape = large_image_stack.shape
    print(shape)
    return large_image_stack, shape

def read_mask(file_id):    
    '''
        This function will take the id of the tiff image and serach for its mask in the training file
        folder. It first opens the json file, use geometry & coordinates fields to make polygons, draw
        the image using the polygons and returns it as the mask
    '''
    json_filename = '../input/hubmap-kidney-segmentation/train/'+file_id+'.json'
    read_file = open(json_filename, "r") 
    data = json.load(read_file)

    polys = []
    for index in range(data.__len__()):
        geom = np.array(data[index]['geometry']['coordinates'])
        polys.append(geom)
    #shape = (38160, 39000)

    Image.MAX_IMAGE_PIXELS = None
    mask = Image.new('L', (shape[1], shape[0]), 0)  # (w, h)
    for i in range(len(polys)):
        poly = polys[i]
        ImageDraw.Draw(mask).polygon(tuple(map(tuple, poly[0])), outline=i + 1, fill=i + 1) 
    polys = []
    mask = np.array(mask)
    return mask

In [ ]:
%%time
files = file_list('train') # to list all files in the train directory
c = 0
patch_length = size_x # tile size
step = 2000 # tile after every this much of pixels
resize = 256 # whole image should be resized to this before saving it
items = 0
for file_id in files[:]:
    large_image_stack, shape = read_tiff(file_id,"train")
    # if needed do the resize as below, but it take lot of resources
    #large_image_stack = cv2.resize(large_image_stack,(shape[1]//3,shape[0]//3))
    print("New shape,",shape)
    # easy tiling of the large image
    patches_img = patchify(large_image_stack, (patch_length, patch_length, 3), step=step)
    # save memory
    large_image_stack = ""
    gc.collect()
    
    # do the same for the mask
    mask = read_mask(file_id)
    patches_mask = patchify(mask, (patch_length, patch_length), step=step)
    mask = ""
    gc.collect()
    
    # calculate the percentage of the mask in the tiled image
    count = 0
    sums = []
    for row in range(patches_mask.shape[0]):
        for col in range(patches_mask.shape[1]):
            s = np.sum(patches_mask[row,col,:,:])
            count+=1
            sums.append([s, row, col, np.round(s/(patch_length*patch_length*255),2)])
    sums.sort()
    #if wanted filter images based on the coverage of the mask
    #ex: first 10% with zero masks + 40-60% mask coverage in the middle + more than 90% mask coverage
    #sums = sums[0:int(len(sums)*.10)]+sums[int(len(sums)*.40):int(len(sums)*.60)]+(sums[int(len(sums)*.90):])
    c = 0
    for im in sums:
        c+=1
        row = im[1]
        col = im[2]

        # seperate corresponding image and its mask using sums list
        image = patches_img[row,col,0,:,:,:]
        image = cv2.resize(image, (resize, resize))
        mask = patches_mask[row,col,:,:]
        mask = cv2.resize(mask, (resize, resize))
        mask = (mask>0).astype(int)
        
        # do augmentaions
        org_img, org_msk, aug_img, aug_msk = augment(image,mask)

        # create uniqe file names
        filename_image = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_image.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"image.jpg"
        cv2.imwrite("./images/"+filename_image, org_img)
        filename_mask = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_mask.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"mask.jpg"
        cv2.imwrite("./masks/"+filename_mask, org_msk*255)
        
        #use following if some randomness is required in saving images
#         num = random.randint(1,100)
#         #print("Num is {}".format(num))
#         if num>=50:
#             filename_image = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_image_aug.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"image.jpg"
#             cv2.imwrite("./images/"+filename_image, aug_img)
#             filename_mask = file_id+"_"+str("{0:0=3d}".format(row))+"_"+str("{0:0=3d}".format(col))+"_mask_aug.jpg"#file_id+"_"+str(str("{0:0=3d}".format(c)))+"mask.jpg"
#             cv2.imwrite("./masks/"+filename_mask, org_msk*255)
        print(".",end="")
            
            #if im[3]>=.30: break
            #print(row, col, filename_image, filename_mask, "saved")
    print()
    print(items," ","Done",c," images")
    print()
    items+=1
    gc.collect()
    
print("DONE=================")

In [ ]:
# Inspeck few images ans its masks
ims = os.listdir('./images')
random.shuffle(ims)
msk = [i.replace("image","mask") for i in ims]
ims = ['./images/'+x for x in ims]
msk = ['./masks/'+x for x in msk]
fig, axs = plt.subplots(1,2,figsize=(10,10))
num = random.randint(1,len(ims))
print(num)
m = cv2.imread(ims[num])
n = cv2.imread(msk[num])
axs[0].imshow(m)
axs[1].imshow(n)

## Now we can use images and its masks to create ML models